## Importing Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

## Data Preprocessing

### Training Image Preprocessing

In [ ]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    verbose=True,
)

### Validation Image Preprocessing

In [ ]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

In [ ]:
training_set

In [ ]:
for x,y in training_set:
    #   x is training example, y is label
    print(x,x.shape)
    print(y,y.shape)
    break

## Building Model

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, InputLayer
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()

## Building Convolution Layer

In [ ]:
model.add(InputLayer(shape=(128, 128, 3)))

In [ ]:
model.add(Conv2D(filters=32,kernel_size=3,padding='same', activation='relu'))
model.add(Conv2D(filters=32,kernel_size=3, activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=64,kernel_size=3,padding='same', activation='relu'))
model.add(Conv2D(filters=64,kernel_size=3, activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=128,kernel_size=3,padding='same', activation='relu'))
model.add(Conv2D(filters=128,kernel_size=3, activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=256,kernel_size=3,padding='same', activation='relu'))
model.add(Conv2D(filters=256,kernel_size=3, activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=512,kernel_size=3,padding='same', activation='relu'))
model.add(Conv2D(filters=512,kernel_size=3, activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Dropout(0.25))  # to avoid overfitting

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(1500,activation='relu'))

In [ ]:
model.add(Dropout(0.4))

In [ ]:
# Output Layer
model.add(Dense(38,activation='softmax'))

## Compiling Model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

### Model Training

In [ ]:
training_history = model.fit(x=training_set, validation_data= validation_set, epochs=5)

### Model Evaluation

In [ ]:
#  Model evaluation on training set
train_loss, train_acc = model.evaluate(training_set)

In [ ]:
print(train_loss, train_acc*100)

In [ ]:
# Model evaluation on Validation Set
val_loss, val_acc = model.evaluate(validation_set)

In [ ]:
print(val_loss, val_acc*100)

### Saving Model

In [ ]:
# model.save('trained_model2.keras')

In [ ]:
export_archive = keras.export.ExportArchive()
export_archive.track(model)
export_archive.write_out("plant_model")

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("plant_model")
tflite_model = converter.convert()

In [ ]:
training_history.history

In [ ]:
epochs = [i for i in range(1, 6)]
plt.plot(epochs, training_history.history['accuracy'], color='red', label='training_accuracy')
plt.plot(epochs, training_history.history['val_accuracy'], color='blue', label='validation_accuracy')
plt.xlabel("no. of epochs")
plt.ylabel("accuracy results")
plt.title("visualization of accuracy result")
plt.legend()
plt.show()

In [ ]:
keras_model = tf.keras.models.load_model("trained_model1.keras")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()


In [ ]:
# import tensorflow as tf

# # Load the saved model
# loaded_model = tf.keras.models.load_model('trained_model.keras')

# # Convert the model to TensorFlow Lite format
# converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
# tflite_model = converter.convert()

# # Save the TensorFlow Lite model to a file
# with open('plant_model.tflite', 'wb') as f:
#     f.write(tflite_model)

# print("Model converted successfully to TensorFlow Lite format!")


In [ ]:
with open("converted_model.tflite", "wb") as f:
    f.write(tflite_model)